In [1]:
import pandas as pd
import numpy as np
import os
%load_ext autoreload
%autoreload 2 

In [2]:
from utils import preprocess_utility_matrix as um
from method1 import build_svd_model as sm

In [3]:
movies_df = pd.DataFrame({'M1': [2, None, 3, None, 1, 5],
                          'M2': [None, 1, None, 5, 1, None],
                          'M3': [5, 4, 1, 1, 4, None],
                          'M4': [5, 4, None, None, None, 1]},
                         index=[f'U{i+1}' for i in range(6)])
movies_df

,M1,M2,M3,M4
U1,2.0,NaN,5.0,5.0
U2,NaN,1.0,4.0,4.0
U3,3.0,NaN,1.0,NaN
U4,NaN,5.0,1.0,NaN
U5,1.0,1.0,4.0,NaN
U6,5.0,NaN,NaN,1.0


# Utility Matrix

In [4]:
U_df = movies_df.copy()
U_df

,M1,M2,M3,M4
U1,2.0,NaN,5.0,5.0
U2,NaN,1.0,4.0,4.0
U3,3.0,NaN,1.0,NaN
U4,NaN,5.0,1.0,NaN
U5,1.0,1.0,4.0,NaN
U6,5.0,NaN,NaN,1.0


In [5]:
U_df_m = um.mean_filled_utilmat(U_df)
U_df_m

,M1,M2,M3,M4
U1,2.0,4.0,5.0,5.0
U2,3.0,1.0,4.0,4.0
U3,3.0,2.0,1.0,2.0
U4,3.0,5.0,1.0,3.0
U5,1.0,1.0,4.0,2.0
U6,5.0,3.0,3.0,1.0


## Build Models

In [6]:
from sklearn.neural_network import MLPClassifier, MLPRegressor
mlp1 = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=300)

In [7]:
U = U_df.copy()
model_object = mlp1

models_item = sm.initialize_models_itemwise(model_object, U, suffix='')
models_item

{'M1': MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=300),
 'M2': MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=300),
 'M3': MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=300),
 'M4': MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=300)}

In [8]:
U_imputed, models = sm.train_model_svd(U, mlp1, d=2, return_models=True)

In [9]:
U_df

,M1,M2,M3,M4
U1,2.0,NaN,5.0,5.0
U2,NaN,1.0,4.0,4.0
U3,3.0,NaN,1.0,NaN
U4,NaN,5.0,1.0,NaN
U5,1.0,1.0,4.0,NaN
U6,5.0,NaN,NaN,1.0


In [10]:
U_imputed

,M1,M2,M3,M4
U1,2.000000,0.795419,5.000000,5.000000
U2,0.834251,1.000000,4.000000,4.000000
U3,3.000000,5.643720,1.000000,-0.503295
U4,3.317594,5.000000,1.000000,-1.351086
U5,1.000000,1.000000,4.000000,3.574352
U6,5.000000,9.026146,-0.815015,1.000000


In [11]:
U_imputed.clip(1,5)

,M1,M2,M3,M4
U1,2.000000,1.0,5.0,5.000000
U2,1.000000,1.0,4.0,4.000000
U3,3.000000,5.0,1.0,1.000000
U4,3.317594,5.0,1.0,1.000000
U5,1.000000,1.0,4.0,3.574352
U6,5.000000,5.0,1.0,1.000000


In [12]:
U_imputed.clip(1,5).round()

,M1,M2,M3,M4
U1,2.0,1.0,5.0,5.0
U2,1.0,1.0,4.0,4.0
U3,3.0,5.0,1.0,1.0
U4,3.0,5.0,1.0,1.0
U5,1.0,1.0,4.0,4.0
U6,5.0,5.0,1.0,1.0


In [13]:
U_df_m

,M1,M2,M3,M4
U1,2.0,4.0,5.0,5.0
U2,3.0,1.0,4.0,4.0
U3,3.0,2.0,1.0,2.0
U4,3.0,5.0,1.0,3.0
U5,1.0,1.0,4.0,2.0
U6,5.0,3.0,3.0,1.0


In [14]:
models

{'M1': MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=300),
 'M2': MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=300),
 'M3': MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=300),
 'M4': MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=300)}